In [1]:
import networkx as nx
import torch
import dgl
import dgl.function as fn
import numpy as np
# ignore potential warnings
import warnings
warnings.filterwarnings("ignore")
import time

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    x = torch.randn(4, 4)
    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
    x = x.to(device)                       # or just use strings ``.to("cuda")``
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))       

In [ ]:
N = 10000
g1 = nx.nx.erdos_renyi_graph(N, 0.05)

In [ ]:
DAMP = 0.85
K = 200

In [ ]:
using the neteworkx librariy and the excellent dgl library from https://docs.dgl.ai/ NYU Shanghai 
by Prof. Zheng Zhang and Quan Gan.  The algorithm below is from their work.

In [ ]:
#a utility that builds a torch-based graph from the nx graph
def makeGraph(g1):
    g = dgl.DGLGraph(g1)
    for i in range(0, N-1):
        g.add_edge(i, i+1)
    g.add_edge(N-1, 0)
    #print(g)
    return g

In [ ]:
def compute_pagerank(g, device):
    g.ndata['pv'] = torch.ones(N).to(device) / N
    degrees = g.out_degrees(g.nodes()).type(torch.float32).to(device)

    for k in range(K):
        g.ndata['pv'] = g.ndata['pv'] / degrees
        g.update_all(message_func=fn.copy_src(src='pv', out='m'),
                     reduce_func=fn.sum(msg='m', out='pv'))
        g.ndata['pv'] = (1 - DAMP) / N + DAMP * g.ndata['pv']
    return g.ndata['pv']/g.ndata['pv'].norm()



The basic page rank algorithm is
$$
Pr(i) ~=~ \frac{(1-d)}{N} ~+~ d \sum_{j\in link(i)}{\frac{ Pr(j)}{out(j)}}
$$

The flowing two versions use the iteration to solve for the Pr vector
$$
Pr_{i+1} ~=~  \frac{(1-d)}{N} ~+~ d G \cdot Out \cdot Pr_{i}
$$
pagerank1 uses the sparse version of the linear algebra.

In [ ]:
def compute_pagerank1(g, K, device):
    pva = torch.ones([N,1])/N
    pv = pva.to(device)
    #print(pv)
    degreesa = 1.0/g.out_degrees(g.nodes()).type(torch.float32)
    degrees = degreesa.reshape([N,1]).to(device)
    edges = g.adjacency_matrix().to(device)
    Idmpa = torch.ones([N,1])*(1-DAMP)/N
    Idmp = Idmpa.to(device)
    #gm = edges.to_dense()
    gm = edges
    for k in range(K):
        pv = pv*degrees
        q = torch.mm(gm, pv)
        pv  = Idmp + DAMP * q
    return pv/pv.norm()


The second version uses the dense representation of the adjacency matrix.

In [ ]:
def compute_pagerank2(g, K, device):
    pv = torch.ones([N]).to(device)/N
    degrees = 1.0/g.out_degrees(g.nodes()).type(torch.float32)
    degrees = degrees.to(device)
    edges = g.adjacency_matrix().to(device)
    Idmp = torch.ones([N])*(1.0-DAMP)/N
    gm = edges.to_dense().to(device)
    for k in range(K):
        pv = pv*degrees
        q = torch.mv(gm, pv)
        pv  = Idmp + DAMP * q
    return pv/pv.norm()



The  third version moves the vector-vector multiply from the innerloop to a matrix matrix multiply outside the loop.

In [ ]:
def compute_pagerank3(g, K, device):
    pv = torch.ones([N]).to(device)/N
    degrees = 1.0/g.out_degrees(g.nodes()).type(torch.float32)
    degrees = degrees.to(device)
    edges = g.adjacency_matrix().to(device)
    Idmp = torch.ones([N]).to(device)*(1.0-DAMP)/N
    gm = edges.to_dense().to(device)
    gm = torch.mm(gm,torch.diag(degrees))
    for k in range(K):
        q = torch.mv(gm, pv)
        pv  = Idmp + DAMP * q
    return pv/pv.norm()


In [ ]:
g = makeGraph(g1)
t0 =time.time() 
pv = compute_pagerank(g, 'cpu')
tser = time.time()-t0
print('elapsed =', tser)

g = makeGraph(g1)
t0 =time.time() 
pvcu= compute_pagerank(g, 'cuda')
tsercu = time.time()-t0
print('elapsed =', tsercu)
print('tser/tsercu=', tser/tsercu)
print(pv)

In [ ]:
g = makeGraph(g1)
t0 =time.time() 
pv1 = compute_pagerank1(g, K, 'cpu')
tv1cpu = time.time()-t0
print('elapsed =', tv1cpu)
print((pv - pv1.T.to('cpu')).norm())

g = makeGraph(g1)
t0 =time.time() 
pv1cu = compute_pagerank1(g, K, 'cuda')
tv1cu =  time.time()-t0
print('elapsed =', tv1cu)
print('cpu/cu=', tv1cpu/tv1cu, ' tser/cu=',tser/tv1cu )
print((pv - pv1cu.T.to('cpu')).norm())
print(pv1)

In [ ]:
g = makeGraph(g1)
t0 =time.time() 
pv2 = compute_pagerank2(g, K, 'cpu')
tv2cpu = time.time()-t0
print('elapsed =', tv2cpu)
print((pv/pv.norm() - pv2.T.to('cpu')).norm())

g = makeGraph(g1)
t0 =time.time() 
pv2cu = compute_pagerank2(g, K, 'cuda')
tv2cu =  time.time()-t0
print('elapsed =', tv2cu)
print('cpu/cu=', tv2cpu/tv2cu, ' tser/cu2=',tser/tv2cu, ' tv1cu/cu2=', tv1cu/tv2cu)
print((pv/pv.norm() - pv2cu.T.to('cpu')).norm())
print(pv2)

In [ ]:
g = makeGraph(g1)
t0 =time.time() 
pv3 = compute_pagerank3(g, K, 'cpu')
print('elapsed =', time.time()-t0)
print((pv - pv3.T.to('cpu')).norm())
print((pv2 - pv3.to('cpu')).norm())
print(pv3)